# CNN 2 - Training image classification models with ANNs

- **Important** - Training image classifiers with plain ANNs isn't a good idea. We're training a model anyway just to see how good of a model we can get. You should never use ANNs for image classification.

- Dataset: 
    - https://www.kaggle.com/pybear/cats-vs-dogs?select=PetImages

- Code: https://github.com/fenago/deeplearning/blob/main/tensorflow/008_CNN_001_Working_With_Image_Data.ipynb
    
- We'll need a quite a bit of imports:

In [ ]:
import os
import pathlib
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image, ImageOps
from IPython.display import display
from sklearn.utils import shuffle
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
warnings.filterwarnings('ignore')

- Let's load in an arbitrary image:

In [ ]:
src_img = Image.open('data/train/cat/1.jpg')
display(src_img)

- Here's the shape (width, height, color channels):

In [ ]:
np.array(src_img).shape

- If flattened, it would result in this many features:

In [ ]:
281 * 300 * 3

- We can reduce the number by a factor of 3 by grayscaling the image
- We still know it's a cat, no matter if we lose the color info:

In [ ]:
gray_img = ImageOps.grayscale(src_img)
display(gray_img)

In [ ]:
np.array(gray_img).shape

In [ ]:
281 * 300

- It's still a lot, so let's resize the image to something smaller
- Let's say 96x96:

In [ ]:
gray_resized_img = gray_img.resize(size=(96, 96))
display(gray_resized_img)

In [ ]:
np.array(gray_resized_img).shape

- Much less features:

In [ ]:
96 * 96

- This is how you can flatten the image and store it as an array:

In [ ]:
np.ravel(gray_resized_img)

- The values aren't in an ideal range (0-255)
- Neural network model prefers 0-1 range
- Let's transform it:

In [ ]:
img_final = np.ravel(gray_resized_img) / 255.0

In [ ]:
img_final

- Finally, let's implement all of this in a single function:

In [ ]:
def process_image(img_path: str) -> np.array:
    img = Image.open(img_path)
    img = ImageOps.grayscale(img)
    img = img.resize(size=(96, 96))
    img = np.ravel(img) / 255.0
    return img

- And let's test it:

In [ ]:
tst_img = process_image(img_path='data/validation/dog/10012.jpg')

In [ ]:
tst_img

In [ ]:
Image.fromarray(np.uint8(tst_img * 255).reshape((96, 96)))

- It works as expected, so let's apply the same logic to the entire dataset next.

<br>

## Process the entire dataset

- Let's declare a function that will process all images in a given folder
- The function returns processed images as a Pandas DataFrame
- We'll add an additional column just so we know the class:

In [ ]:
def process_folder(folder: pathlib.PosixPath) -> pd.DataFrame:
    # We'll store the images here
    processed = []
    
    # For every image in the directory
    for img in folder.iterdir():
        # Ensure JPG
        if img.suffix == '.jpg':
            # Two images failed for whatever reason, so let's just ignore them
            try:
                processed.append(process_image(img_path=str(img)))
            except Exception as _:
                continue
           
    # Convert to pd.DataFrame
    processed = pd.DataFrame(processed)
    # Add a class column - dog or a cat
    processed['class'] = folder.parts[-1]
    
    return processed

- And now let's build ourselves training, validation, and test sets
- We'll start with the training set
    - Process both cat and dog images
    - Concatenate the two datasets
    - Save them in a pickle format, just so you don't have to go through the entire process again

In [ ]:
%%time

train_cat = process_folder(folder=pathlib.Path.cwd().joinpath('data/train/cat'))
train_dog = process_folder(folder=pathlib.Path.cwd().joinpath('data/train/dog'))

train_set = pd.concat([train_cat, train_dog], axis=0)

with open('train_set.pkl', 'wb') as f:
    pickle.dump(train_set, f)

In [ ]:
train_set.head()

In [ ]:
train_set.shape

- Now for the test set:

In [ ]:
%%time

test_cat = process_folder(folder=pathlib.Path.cwd().joinpath('data/test/cat'))
test_dog = process_folder(folder=pathlib.Path.cwd().joinpath('data/test/dog'))

test_set = pd.concat([test_cat, test_dog], axis=0)

with open('test_set.pkl', 'wb') as f:
    pickle.dump(test_set, f)

In [ ]:
test_set.shape

- And finally for the validation set:

In [ ]:
%%time

valid_cat = process_folder(folder=pathlib.Path.cwd().joinpath('data/validation/cat'))
valid_dog = process_folder(folder=pathlib.Path.cwd().joinpath('data/validation/dog'))

valid_set = pd.concat([valid_cat, valid_dog], axis=0)

with open('valid_set.pkl', 'wb') as f:
    pickle.dump(valid_set, f)

In [ ]:
valid_set.shape

<br>

## Additional processing
- Datasets now contain images of cats first, followed by images of dogs
- We want to shuffle those datasets, so a neural network  goes through the images in a random order:

In [ ]:
train_set = shuffle(train_set).reset_index(drop=True)
valid_set = shuffle(valid_set).reset_index(drop=True)

In [ ]:
train_set.head()

- Separate the features from the target:

In [ ]:
X_train = train_set.drop('class', axis=1)
y_train = train_set['class']

X_valid = valid_set.drop('class', axis=1)
y_valid = valid_set['class']

X_test = test_set.drop('class', axis=1)
y_test = test_set['class']

- We need to factorize the target variable
- For example, if our classes are ['cat', 'dog'], the function will convert them to integers [0, 1]
- Then, each instance is represented as follows:
    - Cat: [1, 0]
    - Dog: [0, 1]

In [ ]:
y_train.factorize()

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train.factorize()[0], num_classes=2)
y_valid = tf.keras.utils.to_categorical(y_valid.factorize()[0], num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test.factorize()[0], num_classes=2)

In [ ]:
y_train[:5]

<br>

## Training the model
- The architecture and parameters are completely random
- Set it to whatever you want
- We have two nodes at the output layer
    - Represents two classes - cat and dog
- We're using Categorical Crossentropy as a loss function because we have two categories - cat and dog
- The model is trained for 100 epochs with a batch size of 128:

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=128,
    validation_data=(X_valid, y_valid)
)

<br>

## Inspecting performance
- It doesn't look like the best model, as ANNs aren't the best tool for image data
- Let's visualize training loss vs. validation loss and training accuracy vs. validation accuracy

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = (18, 8)
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

In [ ]:
plt.plot(np.arange(1, 101), history.history['loss'], label='Training Loss')
plt.plot(np.arange(1, 101), history.history['val_loss'], label='Validation Loss')
plt.title('Training vs. Validation Loss', size=20)
plt.xlabel('Epoch', size=14)
plt.legend();

In [ ]:
plt.plot(np.arange(1, 101), history.history['accuracy'], label='Training Accuracy')
plt.plot(np.arange(1, 101), history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs. Validation Accuracy', size=20)
plt.xlabel('Epoch', size=14)
plt.legend();

- The performance is terrible
- 60% accuracy for a binary classifier is almost useless
- Convolutions can help, and you'll see how in the following notebook